<a href="https://colab.research.google.com/github/submouse9903/uos-deepLearning/blob/main/U47768_CH05_RNN(Stock).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(1)
if device == 'cuda':
    torch.cuda.manual_seed_all(1)

In [ ]:
# Load data
df = pd.read_csv('http://ranking.uos.ac.kr/class/RB/stock_data.csv')

In [ ]:
df.head()

,Open,High,Low,Volume,Close
0,828.659973,833.450012,828.349976,1247700,831.659973
1,823.020020,828.070007,821.655029,1597800,828.070007
2,819.929993,824.400024,818.979980,1281700,824.159973
3,819.359985,823.000000,818.469971,1304000,818.979980
4,819.000000,823.000000,816.000000,1053600,820.450012


In [ ]:
x_0 = np.array(df['Close'][0:-1])
x_1 = np.array(df['Close'][1:])
x = np.log(1-(x_1-x_0)/x_0)

In [ ]:
# Define hyperparameters
seq_len = 6
hidden_size = 5
num_layers = 1
learning_rate = 0.001
num_epochs = 100

In [ ]:
rnn = nn.RNN(input_size = 1, hidden_size = 5, num_layers = 1, batch_first = True)

In [ ]:
rnn

RNN(1, 5, batch_first=True)

- Batch_first = False 인 경우에 (seq, batch, feature) 로 input data x 를 입력한다. (Default 값으로 지정되어 있다)
- Batch_first = True 인 경우에 (batch, seq, feature) 로 input data x 를 입력한다. 
- 출력값 역시 Batch_first 옵션을 따른다.


In [ ]:
# 2개 샘플 만들기
print(x[0:6])
print(x[1:7])
x_data = np.concatenate([x[0:6],x[1:7]], axis = 0)
print(x_data)
x_data = x_data.reshape(2,6,1)
input = torch.tensor(x_data).float()
print(input.shape)
print(input.dtype)


[ 0.00430734  0.00471075  0.00626551 -0.00179657  0.00147374  0.00677598]
[ 0.00471075  0.00626551 -0.00179657  0.00147374  0.00677598  0.00503846]
[ 0.00430734  0.00471075  0.00626551 -0.00179657  0.00147374  0.00677598
  0.00471075  0.00626551 -0.00179657  0.00147374  0.00677598  0.00503846]
torch.Size([2, 6, 1])
torch.float32


h0 만들기
- Batch_first = True 와 상관없이 (num_layers*D, batch, hidden_size) 로 h0 를 입력한다.  (만약 bidirectional 이면 D = 2, 아니면 D = 1)
- 위의 예제에서는 2개 샘플, 1개 층, 5차원이므로 (1,2,5)를 입력한다.




In [ ]:
h0 = torch.randn(1, 2, 5)
h0.dtype

torch.float32

In [ ]:
output, hn = rnn(input, h0)
print("output")
print(output)
print(output.shape)
print("hn")
print(hn)
print(hn.shape)

output
tensor([[[ 0.0820, -0.6452,  0.6978,  0.0775,  0.2177],
         [ 0.5276,  0.3945,  0.3164, -0.4088,  0.3101],
         [ 0.4229,  0.0049,  0.4304, -0.5231,  0.3707],
         [ 0.4951,  0.1989,  0.3554, -0.5999,  0.4046],
         [ 0.4725,  0.1331,  0.3805, -0.6150,  0.4069],
         [ 0.4863,  0.1639,  0.3665, -0.6226,  0.4127]],

        [[ 0.3000, -0.0588,  0.6551, -0.7776,  0.2603],
         [ 0.5378,  0.2909,  0.1779, -0.5969,  0.5176],
         [ 0.4386,  0.1031,  0.4482, -0.6515,  0.3629],
         [ 0.4959,  0.1859,  0.3229, -0.6127,  0.4411],
         [ 0.4734,  0.1440,  0.3921, -0.6315,  0.3980],
         [ 0.4870,  0.1622,  0.3581, -0.6215,  0.4205]]],
       grad_fn=<TransposeBackward1>)
torch.Size([2, 6, 5])
hn
tensor([[[ 0.4863,  0.1639,  0.3665, -0.6226,  0.4127],
         [ 0.4870,  0.1622,  0.3581, -0.6215,  0.4205]]],
       grad_fn=<StackBackward0>)
torch.Size([1, 2, 5])


Batch_first=True 이므로 output 의 형태는 (batch, seq, feature) 로 이루어진다. 
- output[0,:,:] 은 첫 번째 input에 대한 rnn feature 행렬인데, 행벡터가 rnn 각 노드에서 hidden feature 에 해당한다.
- hn[:,0,:] 은 관측치 두개에 대한 마지막 hidden feature 값인데, hn[:,0,:] 은 첫번째 input에 대한 hidden feature vector에 해당한다.

In [ ]:
# Define the RNN model
class myRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(myRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.rnn(x, h0)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

In [ ]:
x[0:12]

array([ 0.00430734,  0.00471075,  0.00626551, -0.00179657,  0.00147374,
        0.00677598,  0.00503846,  0.00145651,  0.00174275,  0.00695242,
       -0.00018716,  0.00368627])

In [ ]:
# window size 설정
window_size = 6
# 입력 시퀀스와 출력 시퀀스 정의
def create_inout_sequences(input_data, window_size):
    inout_seq = []
    L = len(input_data)
    for i in range(L - window_size):
        train_seq = input_data[i:(i+window_size)].reshape(window_size,1)
        train_seq = train_seq.astype(np.float32)
        train_label = input_data[(i+window_size):(i+window_size+1)]
        train_label = train_label.astype(np.float32)
        inout_seq.append((train_seq ,train_label))
    return inout_seq

In [ ]:
# 입력 시퀀스와 출력 시퀀스 생성
train_data = create_inout_sequences(x, window_size)
print('input:', train_data[0][0].shape)
print('output:', train_data[0][1].shape)
print('train_data:', len(train_data))

train_data: 725


In [ ]:
# 데이터를 batch 단위로 나누기
batch_size = 10
train_loader = torch.utils.data.DataLoader(train_data, shuffle=False, batch_size=batch_size)

# batch 단위로 데이터 출력
for i, (inputs, labels) in enumerate(train_loader):
    print(f'Batch {i}:')
    #print('Inputs: \n', inputs)
    print(inputs.shape)
    #print('Labels: \n', labels)
    print(labels.shape)
    break

Batch 0:
torch.Size([10, 6, 1])
torch.Size([10, 1])


In [ ]:
model = myRNN(1, hidden_size, num_layers).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for i, (inputs, labels) in enumerate(train_loader):
  input = inputs.to(device)
  label = labels.to(device)
  output = model(input)
  print(output)
  #print(input.shape)
  break 

tensor([[0.0402],
        [0.0403],
        [0.0401],
        [0.0400],
        [0.0402],
        [0.0400],
        [0.0400],
        [0.0401],
        [0.0398],
        [0.0394]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [ ]:
for i, (inputs, labels) in enumerate(train_loader):
  input = inputs.to(device)
  label = labels.to(device)
  output = model(input)
  loss = criterion(output, label)
  print(loss)
  break 

tensor(0.0018, device='cuda:0', grad_fn=<MseLossBackward0>)


In [ ]:
model = myRNN(1, hidden_size, num_layers).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer.zero_grad()

for i, (inputs, labels) in enumerate(train_loader):
  input = inputs.to(device)
  label = labels.to(device)
  output = model(input)
  loss = criterion(output, label)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
print(loss)

tensor(7.0675e-05, device='cuda:0', grad_fn=<MseLossBackward0>)


In [ ]:
model = myRNN(1, hidden_size, num_layers).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer.zero_grad()

num_epochs = 10
for epoch in range(num_epochs):
  for i, (inputs, labels) in enumerate(train_loader):
    input = inputs.to(device)
    label = labels.to(device)
    output = model(input)
    loss = criterion(output, label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print(loss)

tensor(6.6740e-05, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(7.6011e-05, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(7.9406e-05, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(8.1003e-05, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(8.2363e-05, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(8.3481e-05, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(8.4382e-05, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(8.5101e-05, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(8.5666e-05, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(8.6091e-05, device='cuda:0', grad_fn=<MseLossBackward0>)
